## Rating Prediction using Linear Regression

In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
np.set_printoptions(precision=2)
pd.set_option('display.precision', 2)

In [3]:
movies = pd.read_csv('data/movies_w_imgurl.csv')

In [4]:
movieGenres = pd.DataFrame(data=movies['genres'].str.split('|').apply(pd.Series, 1).stack(), columns=['genre'])
movieGenres.index = movieGenres.index.droplevel(1)

In [5]:
genres = movieGenres.groupby('genre').count()

In [6]:
movieWeights = pd.DataFrame(data=movies['movieId'])
for genre in genres.index:
    df = pd.DataFrame(data=movieGenres[movieGenres['genre']==genre], columns=[genre])
    df[genre] = 1
    movieWeights = movieWeights.join(df, on='movieId')
movieWeights.fillna(0, inplace=True)
movieWeights

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9121,163056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9122,163949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,164977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Make Regression Model for Users

In [7]:
ratings = pd.read_csv('data/ratings-9_1.csv')
train = ratings[ratings['type'] == 'train'][['userId', 'movieId', 'rating']]
test = ratings[ratings['type'] == 'test'][['userId', 'movieId', 'rating']]

In [8]:
users = [33, 39, 77, 144, 238]

In [9]:
userId = 33
userRatings = train[train['userId'] == userId][['movieId', 'rating']]
userRatings = userRatings.sort_values(by='movieId')

In [10]:
userLRTrain = movieWeights[movieWeights['movieId'].isin(userRatings['movieId'].values)].sort_values(by='movieId')

In [11]:
X = userLRTrain.iloc[:,1:].values
Y = userRatings['rating'].values

In [12]:
from sklearn import linear_model as lm

In [13]:
reg = lm.LinearRegression()

In [14]:
reg.fit(X, Y)

LinearRegression()

In [15]:
print(reg.coef_)
print(reg.intercept_)

[ 0.00e+00  3.36e-01 -1.37e-01 -3.08e-02  3.13e-01 -3.07e-01 -5.42e-02
 -2.26e-01 -6.83e-01  9.10e-03  0.00e+00 -2.62e-01  8.33e-17  3.43e-01
  2.25e-01  2.03e-01 -5.54e-02  1.94e-02  9.46e-02  2.09e-01]
3.559727006488379


In [16]:
userTestRatings = pd.DataFrame(test[test['userId'] == userId])
userTestRatings

,userId,movieId,rating
6187,33,1060,4.0
6198,33,1291,4.0
6199,33,1347,2.0
6208,33,1982,4.0
6212,33,2005,4.0
6215,33,2064,5.0
6257,33,3794,4.0
6292,33,4678,3.0
6303,33,4974,3.0


Make Prediction for Test Movies

In [17]:
pred = reg.predict(movieWeights[movieWeights['movieId'].isin(userTestRatings['movieId'].values)].iloc[:,1:].values)

In [18]:
userTestRatings['prediction'] = pd.Series(data=pred, index = userTestRatings.index)
userTestRatings

,userId,movieId,rating,prediction
6187,33,1060,4.0,3.46
6198,33,1291,4.0,2.88
6199,33,1347,2.0,2.97
6208,33,1982,4.0,3.99
6212,33,2005,4.0,2.57
6215,33,2064,5.0,3.22
6257,33,3794,4.0,3.49
6292,33,4678,3.0,2.88
6303,33,4974,3.0,3.63


In [19]:
mae = (userTestRatings['rating'] - userTestRatings['prediction']).abs().mean()
rmse = math.sqrt((userTestRatings['rating'] - userTestRatings['prediction']).pow(2).mean())
print(" MAE:", mae)
print("RMSE:", rmse)

 MAE: 0.7909124296018574
RMSE: 0.9650362717424515
